In [18]:
import pandas as pd
import numpy as np

In [2]:
organes = (pd.read_csv('out/organes.csv'))
votes = pd.read_csv('out/votes.csv')

In [23]:
alignements = (
    votes
    .groupby(['scrutin', 'organe'])
    .agg({
        'groupe_majorite': 'first'
    })
    .reset_index()
    .assign(
        organe = lambda df: df.join(organes.set_index('uid'), on='organe')[['libelleAbrege']]
    )
    .pivot_table(
        index="scrutin",
        columns="organe",
        values="groupe_majorite",
        aggfunc= lambda x: x
    )
)

alignements

organe,Agir ens,Dem,EDS,FI,GDR,LC,LR,LT,LaREM,MODEM,NG,NI,SOC,UDI-AGIR,UDI-I
scrutin,,,,,,,,,,,,,,,
VTANR5L15V1,NaN,NaN,NaN,contre,contre,abstention,abstention,NaN,pour,pour,abstention,contre,NaN,NaN,NaN
VTANR5L15V10,NaN,NaN,NaN,pour,pour,contre,contre,NaN,contre,contre,pour,contre,NaN,NaN,NaN
VTANR5L15V100,NaN,NaN,NaN,pour,pour,contre,pour,NaN,contre,contre,pour,pour,NaN,NaN,NaN
VTANR5L15V1000,NaN,NaN,NaN,pour,abstention,NaN,contre,NaN,contre,contre,abstention,contre,NaN,abstention,NaN
VTANR5L15V1001,NaN,NaN,NaN,contre,contre,NaN,pour,NaN,contre,contre,pour,pour,NaN,pour,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VTANR5L15V995,NaN,NaN,NaN,contre,contre,NaN,abstention,NaN,pour,pour,abstention,contre,NaN,pour,NaN
VTANR5L15V996,NaN,NaN,NaN,pour,pour,NaN,abstention,NaN,pour,pour,pour,contre,NaN,pour,NaN
VTANR5L15V997,NaN,NaN,NaN,pour,pour,NaN,pour,NaN,contre,contre,pour,pour,NaN,abstention,NaN


In [24]:
def get_covotes(df, groupe):
    return (
        df
        .T
        .fillna('non votant')
        .pipe( lambda df: (
            df
            .eq(df.loc[groupe])
        ))
        .sum(axis=1)
        .to_frame()
        .rename(columns={0: groupe})
    )

covotes = pd.concat([ get_covotes(alignements, g).T for g in alignements.T.index ])

covotes

organe,Agir ens,Dem,EDS,FI,GDR,LC,LR,LT,LaREM,MODEM,NG,NI,SOC,UDI-AGIR,UDI-I
Agir ens,4417,3915,2959,672,764,2587,779,2105,1276,299,1852,1081,1807,786,3338
Dem,3915,4417,2786,566,664,2578,705,1979,1456,68,1843,991,1687,777,3188
EDS,2959,2786,4417,582,617,3918,185,1967,67,1566,3183,749,1439,2121,3010
FI,672,566,582,4417,3162,537,1741,1590,674,707,1167,1945,2206,951,787
GDR,764,664,617,3162,4417,602,1970,1752,758,764,1239,2088,2438,1017,1000
LC,2587,2578,3918,537,602,4417,350,1569,165,1781,3497,836,986,1998,2694
LR,779,705,185,1741,1970,350,4417,1587,1552,1148,591,2562,1629,1281,1054
LT,2105,1979,1967,1590,1752,1569,1587,4417,830,839,834,1676,2904,1431,2278
LaREM,1276,1456,67,674,758,165,1552,830,4417,2501,228,1019,815,1126,689
MODEM,299,68,1566,707,764,1781,1148,839,2501,4417,1897,932,541,2760,580


In [25]:
organes_pca = pd.read_csv('out/organes_pca.csv')

organes_pca

,libelle,axe 1,axe 2,@xmlns,@xmlns:xsi,@xsi:type,codeType,libelleEdition,libelleAbrege,libelleAbrev,...,regime,legislature,regimeJuridique,siteInternet,nombreReunionsAnnuelles,secretariat,listePays,positionPolitique,preseance,couleurAssociee
0,La France insoumise,19.647151,-10.855444,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe La France insoumise,FI,FI,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,7.0,#ED302C
1,Gauche démocrate et républicaine,16.150724,-7.537916,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe de la Gauche démocrate et républicaine,GDR,GDR,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,9.0,#A41914
2,Socialistes et apparentés,13.163086,-4.619929,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Socialistes et apparentés,SOC,SOC,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,4.0,#D46CA9
3,Nouvelle Gauche,10.270789,-4.087692,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Nouvelle Gauche,NG,NG,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,5.0,NaN
4,Écologie Démocratie Solidarité,-3.630505,-3.834768,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Écologie Démocratie Solidarité,EDS,EDS,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,8.0,#3A673E
5,Libertés et Territoires,10.302164,-2.018121,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Libertés et Territoires,LT,LT,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,8.0,#66BF74
6,Mouvement Démocrate (MoDem) et Démocrates appa...,-1.151259,-1.832461,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Mouvement Démocrate (MoDem) et Démoc...,Dem,DEM,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,3.0,#F26D22
7,Non inscrit,12.355559,-1.227555,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,des députés non inscrits,NI,NI,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,99.0,#8D949A
8,Agir ensemble,-2.458028,-0.858535,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Agir ensemble,Agir ens,AGIR-E,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,5.0,#C7C034
9,Mouvement Démocrate et apparentés,-6.624225,-0.768921,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe du Mouvement Démocrate et apparentés,MODEM,MODEM,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,3.0,#F26D22


In [26]:
organes_pca.drop_duplicates()

,libelle,axe 1,axe 2,@xmlns,@xmlns:xsi,@xsi:type,codeType,libelleEdition,libelleAbrege,libelleAbrev,...,regime,legislature,regimeJuridique,siteInternet,nombreReunionsAnnuelles,secretariat,listePays,positionPolitique,preseance,couleurAssociee
0,La France insoumise,19.647151,-10.855444,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe La France insoumise,FI,FI,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,7.0,#ED302C
1,Gauche démocrate et républicaine,16.150724,-7.537916,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe de la Gauche démocrate et républicaine,GDR,GDR,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,9.0,#A41914
2,Socialistes et apparentés,13.163086,-4.619929,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Socialistes et apparentés,SOC,SOC,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,4.0,#D46CA9
3,Nouvelle Gauche,10.270789,-4.087692,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Nouvelle Gauche,NG,NG,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,5.0,NaN
4,Écologie Démocratie Solidarité,-3.630505,-3.834768,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Écologie Démocratie Solidarité,EDS,EDS,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,8.0,#3A673E
5,Libertés et Territoires,10.302164,-2.018121,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Libertés et Territoires,LT,LT,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Opposition,8.0,#66BF74
6,Mouvement Démocrate (MoDem) et Démocrates appa...,-1.151259,-1.832461,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Mouvement Démocrate (MoDem) et Démoc...,Dem,DEM,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,3.0,#F26D22
7,Non inscrit,12.355559,-1.227555,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,des députés non inscrits,NI,NI,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,NaN,99.0,#8D949A
8,Agir ensemble,-2.458028,-0.858535,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe Agir ensemble,Agir ens,AGIR-E,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,5.0,#C7C034
9,Mouvement Démocrate et apparentés,-6.624225,-0.768921,http://schemas.assemblee-nationale.fr/referentiel,http://www.w3.org/2001/XMLSchema-instance,GroupePolitique_type,GP,du groupe du Mouvement Démocrate et apparentés,MODEM,MODEM,...,5ème République,15.0,NaN,NaN,NaN,"{'secretaire01': None, 'secretaire02': None}",NaN,Minoritaire,3.0,#F26D22


In [32]:
(
    covotes[organes_pca.libelleAbrege].loc[organes_pca.libelleAbrege]

)

organe,FI,GDR,SOC,NG,EDS,LT,Dem,NI,Agir ens,MODEM,UDI-AGIR,LaREM,UDI-I,LC,UDI-AGIR,LR
FI,4417,3162,2206,1167,582,1590,566,1945,672,707,951,674,787,537,951,1741
GDR,3162,4417,2438,1239,617,1752,664,2088,764,764,1017,758,1000,602,1017,1970
SOC,2206,2438,4417,251,1439,2904,1687,1698,1807,541,1069,815,1946,986,1069,1629
NG,1167,1239,251,4417,3183,834,1843,1164,1852,1897,2239,228,1959,3497,2239,591
EDS,582,617,1439,3183,4417,1967,2786,749,2959,1566,2121,67,3010,3918,2121,185
LT,1590,1752,2904,834,1967,4417,1979,1676,2105,839,1431,830,2278,1569,1431,1587
Dem,566,664,1687,1843,2786,1979,4417,991,3915,68,777,1456,3188,2578,777,705
NI,1945,2088,1698,1164,749,1676,991,4417,1081,932,1231,1019,1304,836,1231,2562
Agir ens,672,764,1807,1852,2959,2105,3915,1081,4417,299,786,1276,3338,2587,786,779
MODEM,707,764,541,1897,1566,839,68,932,299,4417,2760,2501,580,1781,2760,1148


In [44]:
(
    covotes[organes_pca.libelleAbrege].loc[organes_pca.libelleAbrege]
    # .reset_index(drop=True)
    # .style
    #     .background_gradient(
    #         # cmap='PiYG',
    #     )
)

organe,FI,GDR,SOC,NG,EDS,LT,Dem,NI,Agir ens,MODEM,UDI-AGIR,LaREM,UDI-I,LC,UDI-AGIR,LR
FI,4417,3162,2206,1167,582,1590,566,1945,672,707,951,674,787,537,951,1741
GDR,3162,4417,2438,1239,617,1752,664,2088,764,764,1017,758,1000,602,1017,1970
SOC,2206,2438,4417,251,1439,2904,1687,1698,1807,541,1069,815,1946,986,1069,1629
NG,1167,1239,251,4417,3183,834,1843,1164,1852,1897,2239,228,1959,3497,2239,591
EDS,582,617,1439,3183,4417,1967,2786,749,2959,1566,2121,67,3010,3918,2121,185
LT,1590,1752,2904,834,1967,4417,1979,1676,2105,839,1431,830,2278,1569,1431,1587
Dem,566,664,1687,1843,2786,1979,4417,991,3915,68,777,1456,3188,2578,777,705
NI,1945,2088,1698,1164,749,1676,991,4417,1081,932,1231,1019,1304,836,1231,2562
Agir ens,672,764,1807,1852,2959,2105,3915,1081,4417,299,786,1276,3338,2587,786,779
MODEM,707,764,541,1897,1566,839,68,932,299,4417,2760,2501,580,1781,2760,1148


In [30]:
(
    covotes[organes_pca.libelleAbrege].loc[organes_pca.libelleAbrege]
    .div(
        len(alignements.index), axis=0
    )
    .replace(1, np.nan)
    .style
        .format('{:,.1%}'.format)
    #     .background_gradient(
    #         cmap='PiYG',
    #         # vmin=0,
    #         # vmax=1
    #         axis=1
    #     )
)

organe,FI,GDR,SOC,NG,EDS,LT,Dem,NI,Agir ens,MODEM,UDI-AGIR,LaREM,UDI-I,LC,UDI-AGIR,LR
FI,nan%,71.6%,49.9%,26.4%,13.2%,36.0%,12.8%,44.0%,15.2%,16.0%,21.5%,15.3%,17.8%,12.2%,21.5%,39.4%
GDR,71.6%,nan%,55.2%,28.1%,14.0%,39.7%,15.0%,47.3%,17.3%,17.3%,23.0%,17.2%,22.6%,13.6%,23.0%,44.6%
SOC,49.9%,55.2%,nan%,5.7%,32.6%,65.7%,38.2%,38.4%,40.9%,12.2%,24.2%,18.5%,44.1%,22.3%,24.2%,36.9%
NG,26.4%,28.1%,5.7%,nan%,72.1%,18.9%,41.7%,26.4%,41.9%,42.9%,50.7%,5.2%,44.4%,79.2%,50.7%,13.4%
EDS,13.2%,14.0%,32.6%,72.1%,nan%,44.5%,63.1%,17.0%,67.0%,35.5%,48.0%,1.5%,68.1%,88.7%,48.0%,4.2%
LT,36.0%,39.7%,65.7%,18.9%,44.5%,nan%,44.8%,37.9%,47.7%,19.0%,32.4%,18.8%,51.6%,35.5%,32.4%,35.9%
Dem,12.8%,15.0%,38.2%,41.7%,63.1%,44.8%,nan%,22.4%,88.6%,1.5%,17.6%,33.0%,72.2%,58.4%,17.6%,16.0%
NI,44.0%,47.3%,38.4%,26.4%,17.0%,37.9%,22.4%,nan%,24.5%,21.1%,27.9%,23.1%,29.5%,18.9%,27.9%,58.0%
Agir ens,15.2%,17.3%,40.9%,41.9%,67.0%,47.7%,88.6%,24.5%,nan%,6.8%,17.8%,28.9%,75.6%,58.6%,17.8%,17.6%
MODEM,16.0%,17.3%,12.2%,42.9%,35.5%,19.0%,1.5%,21.1%,6.8%,nan%,62.5%,56.6%,13.1%,40.3%,62.5%,26.0%
